In [17]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader

In [18]:
data = pd.read_csv('dataset_eurousd.csv', delimiter = '\t')
data.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2012.04.09,00:00:00,1.30961,1.30965,1.30662,1.30687,2151,0,13
1,2012.04.09,01:00:00,1.30688,1.30861,1.30668,1.30847,1490,0,12
2,2012.04.09,02:00:00,1.30848,1.30876,1.30441,1.30443,2226,0,12
3,2012.04.09,03:00:00,1.30444,1.30524,1.30330,1.30508,2322,0,13
4,2012.04.09,04:00:00,1.30509,1.30618,1.30504,1.30581,1379,0,12


In [19]:
data.columns =['DATE', 'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL',
       'VOL', 'SPREAD']
data.columns

Index(['DATE', 'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL',
       'SPREAD'],
      dtype='object')

In [20]:
len(data)

62002

In [21]:
n_steps = 24
pred_len = 12

def split_sequence(sequence, n_steps, pred_len):
    x, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        # print(sequence)

        seq_y = sequence.iloc[end_ix:end_ix + pred_len, 5]
        seq_y = seq_y.values
        # print(seq_y.shape)
        seq_x = sequence[i:end_ix].drop(["DATE","TIME","SPREAD"], axis = 1)
        seq_x = seq_x[i:end_ix]
        # print(pca_x.shape)
        if len(seq_x) == n_steps and len(seq_y) == pred_len:
            x.append(seq_x)
            y.append(seq_y)
    return np.array(x), np.array(y)

In [22]:
timeseries_dataset = split_sequence(data[:100], n_steps, pred_len)
# data2 = timeseries_dataset[0]
# data2[0].shape

In [25]:
data_loader = DataLoader(timeseries_dataset, batch_size = 1)


In [8]:
print(timeseries_dataset[1])

[[1.31126 1.31084 1.31165 1.31373 1.31394 1.31311 1.31323 1.31208 1.30897
  1.30848 1.30878 1.30749]]


In [33]:
lstm = nn.LSTM(6, 12, 1, batch_first = True)
print(lstm)
for data in data_loader:
    data2 = data[0]
    output = lstm(double(data2))
    print(output)
    print(output.shape)

LSTM(6, 12, batch_first=True)


NameError: name 'double' is not defined

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)